## 0.导入常用库

In [1]:
# 加入下面2行，可以使py代码文件中的修改即时生效
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

/home/zongyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zongyi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/zongyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zongyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## 1.数据准备

In [3]:
# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------

### 1.1 定义函数load_train_dataset、load_test_dataset，用于加载数据

In [4]:
from keras.datasets.cifar import load_batch

# 加载数据集cifar10里面的训练集
def load_train_dataset(dirPath='../resources/cifar-10-batches-py/'):
    train_sample_quantity = 50000
    image_width = 32
    image_height = 32
    channel_quantity = 3
    train_x = np.zeros((train_sample_quantity, channel_quantity, image_width, image_height),
                       dtype='uint8')
    train_y = np.zeros((train_sample_quantity, ),
                       dtype='uint8')
    for i in range(1, 6):
        fileName = 'data_batch_%d' %i
        filePath = os.path.join(dirPath, fileName)
        startIndex = (i - 1) * 10000
        endIndex = i * 10000
        train_x[startIndex:endIndex, :, :, :], train_y[startIndex:endIndex] = load_batch(filePath)
    print('未做矩阵转置：', train_x.shape)
    # 从官网上下载的数据集的4个维度为样本个数n、通道数c、宽度w、高度h
    # Keras基于Tensorflow，数据的维度顺序要求：样本个数n、宽度w、高度h、通道数c，所以使用np.transpose完成矩阵转置
    train_x = train_x.transpose(0, 2, 3, 1)
    print('矩阵转置后：', train_x.shape)
    return train_x, train_y

# 加载数据集cifar10里面的测试集
def load_test_dataset(dirPath='../resources/cifar-10-batches-py/'):
    fileName = 'test_batch'
    filePath = os.path.join(dirPath, fileName)
    test_x, test_y = load_batch(filePath)
    print('未做矩阵转置：', test_x.shape)
    test_x = test_x.transpose(0, 2, 3, 1)
    print('矩阵转置后：', test_x.shape)
    return test_x, test_y

### 1.2 对加载后的图像数据做处理

In [5]:
# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Load the CIFAR10 data.
dirPath = '../resources/cifar-10-batches-py/'
x_train, y_train = load_train_dataset(dirPath)
x_test, y_test = load_test_dataset(dirPath)

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
num_classes = 10
print('未做to_categorical之前, y_train shape:', y_train.shape)
y_train = keras.utils.to_categorical(y_train, num_classes)
print('做to_categorical之后, y_train shape:', y_train.shape)
y_test = keras.utils.to_categorical(y_test, num_classes)

未做矩阵转置： (50000, 3, 32, 32)
矩阵转置后： (50000, 32, 32, 3)
未做矩阵转置： (10000, 3, 32, 32)
矩阵转置后： (10000, 32, 32, 3)
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000,)
50000 train samples
10000 test samples
未做to_categorical之前, y_train shape: (50000,)
做to_categorical之后, y_train shape: (50000, 10)


## 2.搭建神经网络

### 2.1 定义函数resnet_layer，返回值是经过resnet_layer计算的结果

In [6]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

### 2.2 定义函数resnet_v1，返回值是模型对象

In [7]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]
    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

### 2.3 定义函数resnet_v2，返回值是模型对象

In [8]:
def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

### 2.4 实例化模型对象

In [9]:
# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2

# Computed depth from supplied model parameter n
n = 6
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2
    
# 根据ResNet版本，获取对应的模型对象
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

### 2.5 多GPU并行训练

In [10]:
from parallel_model import ParallelModel

gpu_count = 2
model = ParallelModel(model, gpu_count)

### 2.6 模型部署到GPU上

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

### 2.7 打印模型架构信息

In [12]:
# Model name, depth and version

model_type = 'ResNet%dv%d' % (depth, version)
print(model_type)
model.summary()

ResNet56v2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 32, 32, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 32, 32, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 10)           1673738     lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   

## 3.模型训练

### 3.1 规划学习率，训练到后期，学习率需要减少

In [13]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

### 3.2 模型训练时的参数设置

In [14]:
# Training parameters
batch_size = 64  # orig paper trained all networks with batch_size=128
epochs = 200

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
callbacks = [checkpoint, lr_reducer, lr_scheduler]

### 3.3 使用图像增强的结果做模型训练

In [15]:
data_augmentation = True
if data_augmentation:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs,
                        verbose=1, 
                        workers=4,
                        callbacks=callbacks)

Using real-time data augmentation.
Epoch 1/200
Learning rate:  0.001
782/782 [==============================] - 63s 81ms/step - loss: 2.1703 - acc: 0.4919 - val_loss: 1.9985 - val_acc: 0.5165
Epoch 2/200
Learning rate:  0.001
782/782 [==============================] - 48s 61ms/step - loss: 1.5300 - acc: 0.6350 - val_loss: 1.5223 - val_acc: 0.6298
Epoch 3/200
Learning rate:  0.001
782/782 [==============================] - 49s 62ms/step - loss: 1.2784 - acc: 0.6979 - val_loss: 1.2925 - val_acc: 0.6904
Epoch 4/200
Learning rate:  0.001
782/782 [==============================] - 49s 62ms/step - loss: 1.1373 - acc: 0.7366 - val_loss: 1.1915 - val_acc: 0.7150
Epoch 5/200
Learning rate:  0.001
 29/782 [>.............................] - ETA: 44s - loss: 1.0797 - acc: 0.7559

KeyboardInterrupt: 

In [16]:
batch_size = 128
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs,
                        verbose=1, 
                        workers=4,
                        callbacks=callbacks)

Epoch 1/200
Learning rate:  0.001
391/391 [==============================] - 31s 78ms/step - loss: 0.9425 - acc: 0.7962 - val_loss: 1.0174 - val_acc: 0.7597
Epoch 2/200
Learning rate:  0.001
391/391 [==============================] - 30s 77ms/step - loss: 0.8958 - acc: 0.8030 - val_loss: 1.0530 - val_acc: 0.7540
Epoch 3/200
Learning rate:  0.001
391/391 [==============================] - 30s 78ms/step - loss: 0.8605 - acc: 0.8114 - val_loss: 1.1938 - val_acc: 0.7318
Epoch 4/200
Learning rate:  0.001
391/391 [==============================] - 31s 78ms/step - loss: 0.8215 - acc: 0.8233 - val_loss: 0.9830 - val_acc: 0.7689
Epoch 5/200
Learning rate:  0.001
391/391 [==============================] - 31s 79ms/step - loss: 0.7906 - acc: 0.8308 - val_loss: 1.1271 - val_acc: 0.7368
Epoch 6/200
Learning rate:  0.001
 94/391 [======>.......................] - ETA: 22s - loss: 0.7841 - acc: 0.8306

KeyboardInterrupt: 

In [17]:
batch_size = 256
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs,
                        verbose=1, 
                        workers=4,
                        callbacks=callbacks)

Epoch 1/200
Learning rate:  0.001
196/196 [==============================] - 24s 122ms/step - loss: 0.6848 - acc: 0.8637 - val_loss: 0.8849 - val_acc: 0.7977
Epoch 2/200
Learning rate:  0.001
196/196 [==============================] - 24s 121ms/step - loss: 0.6588 - acc: 0.8678 - val_loss: 1.0683 - val_acc: 0.7365
Epoch 3/200
Learning rate:  0.001
196/196 [==============================] - 24s 123ms/step - loss: 0.6402 - acc: 0.8734 - val_loss: 1.0007 - val_acc: 0.7722
Epoch 4/200
Learning rate:  0.001
196/196 [==============================] - 24s 123ms/step - loss: 0.6248 - acc: 0.8756 - val_loss: 0.9805 - val_acc: 0.7801
Epoch 5/200
Learning rate:  0.001
196/196 [==============================] - 24s 124ms/step - loss: 0.6166 - acc: 0.8772 - val_loss: 0.7843 - val_acc: 0.8224
Epoch 6/200
Learning rate:  0.001
196/196 [==============================] - 24s 124ms/step - loss: 0.6067 - acc: 0.8795 - val_loss: 0.8058 - val_acc: 0.8148
Epoch 7/200
Learning rate:  0.001
196/196 [=========

196/196 [==============================] - 24s 121ms/step - loss: 0.3766 - acc: 0.9434 - val_loss: 0.8503 - val_acc: 0.8283
Epoch 53/200
Learning rate:  0.001
196/196 [==============================] - 24s 120ms/step - loss: 0.3767 - acc: 0.9425 - val_loss: 0.6786 - val_acc: 0.8675
Epoch 54/200
Learning rate:  0.001
196/196 [==============================] - 23s 119ms/step - loss: 0.3759 - acc: 0.9425 - val_loss: 0.7357 - val_acc: 0.8516
Epoch 55/200
Learning rate:  0.001
196/196 [==============================] - 23s 119ms/step - loss: 0.3712 - acc: 0.9453 - val_loss: 0.6621 - val_acc: 0.8679
Epoch 56/200
Learning rate:  0.001
196/196 [==============================] - 23s 119ms/step - loss: 0.3667 - acc: 0.9460 - val_loss: 0.7582 - val_acc: 0.8521
Epoch 57/200
Learning rate:  0.001
196/196 [==============================] - 23s 119ms/step - loss: 0.3688 - acc: 0.9446 - val_loss: 0.7614 - val_acc: 0.8385
Epoch 58/200
Learning rate:  0.001
196/196 [==============================] - 23s

196/196 [==============================] - 23s 118ms/step - loss: 0.1861 - acc: 0.9952 - val_loss: 0.4957 - val_acc: 0.9231
Epoch 104/200
Learning rate:  0.0001
196/196 [==============================] - 23s 118ms/step - loss: 0.1855 - acc: 0.9946 - val_loss: 0.4881 - val_acc: 0.9266
Epoch 105/200
Learning rate:  0.0001
196/196 [==============================] - 23s 119ms/step - loss: 0.1837 - acc: 0.9950 - val_loss: 0.5001 - val_acc: 0.9243
Epoch 106/200
Learning rate:  0.0001
196/196 [==============================] - 23s 118ms/step - loss: 0.1800 - acc: 0.9956 - val_loss: 0.4779 - val_acc: 0.9279
Epoch 107/200
Learning rate:  0.0001
196/196 [==============================] - 23s 118ms/step - loss: 0.1791 - acc: 0.9950 - val_loss: 0.4881 - val_acc: 0.9234
Epoch 108/200
Learning rate:  0.0001
196/196 [==============================] - 23s 119ms/step - loss: 0.1782 - acc: 0.9950 - val_loss: 0.4930 - val_acc: 0.9252
Epoch 109/200
Learning rate:  0.0001
196/196 [=========================

196/196 [==============================] - 24s 122ms/step - loss: 0.1451 - acc: 0.9989 - val_loss: 0.4699 - val_acc: 0.9312
Epoch 155/200
Learning rate:  1e-05
196/196 [==============================] - 24s 120ms/step - loss: 0.1451 - acc: 0.9987 - val_loss: 0.4681 - val_acc: 0.9310
Epoch 156/200
Learning rate:  1e-05
196/196 [==============================] - 24s 121ms/step - loss: 0.1452 - acc: 0.9986 - val_loss: 0.4697 - val_acc: 0.9301
Epoch 157/200
Learning rate:  1e-05
196/196 [==============================] - 24s 121ms/step - loss: 0.1451 - acc: 0.9985 - val_loss: 0.4684 - val_acc: 0.9307
Epoch 158/200
Learning rate:  1e-05
196/196 [==============================] - 24s 120ms/step - loss: 0.1446 - acc: 0.9988 - val_loss: 0.4700 - val_acc: 0.9299
Epoch 159/200
Learning rate:  1e-05
196/196 [==============================] - 24s 121ms/step - loss: 0.1453 - acc: 0.9984 - val_loss: 0.4692 - val_acc: 0.9321
Epoch 160/200
Learning rate:  1e-05
196/196 [==============================]

## 4.模型评估

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])